In [1]:
import os
from typing import List, Optional
from langchain.schema import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain.vectorstores import Chroma 
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb
from groq import Groq
from IPython.display import display, Markdown


In [2]:
class DocumentLoader:
    @staticmethod
    def load_docs(file_paths: List[str]) -> List[Document]:
        """
        ໂຫລດ PDF documents ໂດຍໃຊ້ LangChain PyPDFLoader ເພາະຍັງແອັດຈັງມັກເພາະມັນສ້າງ Metadata ໃຫ້ Auto 

        Metadata ຄືຍັງ ? 
        Metadata ຄືຂໍ້ມູນເພີ່ມເຕີມກ່ຽວກັບເອກະສານ
        ແຕ່ລະ Document ຈະມີ 2 ສ່ວນຫຼັກ:
        1. page_content: ເນື້ອໃນຂໍ້ຄວາມຈິງໆ
        2. metadata: ຂໍ້ມູນລາຍລະອຽດກ່ຽວກັບເອກະສານ ເພື່ອບົງບອກວ່າ ເວລາເຮົາ ຄົ້ນຫາຂໍ້ມູນ ແຫຼ່ງຂໍ້ມູນນັ້ນມາຈາກໄສ
        
        Args:
            file_paths (list): List of PDF file paths
        
        Returns:
            List[Document]: List of LangChain Document objects
        """
        
        all_docs = []
        
        for file_path in file_paths:
            if not os.path.exists(file_path):
                print(f"Warning: File {file_path} not found. Skipping...")
                continue
                
            try:
                file_extension = os.path.splitext(file_path)[1].lower()
                
                # Check if file is PDF
                if file_extension != '.pdf':
                    print(f"Warning: {file_path} is not a PDF file. Skipping...")
                    continue
                
                # Load PDF using LangChain PyPDFLoader
                loader = PyPDFLoader(file_path)
                documents = loader.load()
                
                # Add enhanced metadata to all documents
                for doc in documents:
                    if doc.metadata is None:
                        doc.metadata = {}
                        
                    doc.metadata.update({
                        'source_file': os.path.basename(file_path),
                        'file_type': file_extension,
                        'file_path': file_path,
                        'file_size': os.path.getsize(file_path) if os.path.exists(file_path) else 0,
                    })
                
                all_docs.extend(documents)
                print(f"✅ Processed PDF: {file_path} ({len(documents)} pages)")
                
            except Exception as e:
                print(f"❌ Error processing {file_path}: {str(e)}")
                continue
        
        print(f"📚 Total PDF documents loaded: {len(all_docs)}")
        return all_docs
    
    @staticmethod
    def chunk_documents_standard(
        docs: List[Document], 
        chunk_size: int = 1000,
        chunk_overlap: int = 200,
        tokenizer_model: str = "D:/model/BAAI-bge-m3",
        max_token_limit: int = 8192
    ) -> List[Document]:
        """
        ໃຊ້ Lanchain ໃນການເຮັດ chunking ຂໍ້ມູນ ເພື່ອການຄົ້ນຫາຂໍ້ມູນດ້ວຍ ChromaDB  

        Chunk_size: ແມ່ນຈຳນວນຂໍ້ມູນທີ່ຈະເຮັດ chunking ຕໍ່ຫນ່ວຍ ເພາະຍັງ ເຮົາບໍ່ສາມາດເອົາເອກະສານທັ້ງໝົດໃຫ້ AI ຕອບໄດ້ ເນື່ອງຈາກບາງເອກະສານມີຫລາຍຫນ້າ
        Chunk_overlap: ແມ່ນຈຳນວນຂໍ້ມູນທີ່ຈະເຮັດ chunking ຕໍ່ຫນ່ວຍ ເພາະຍັງ ເຮົາບໍ່ສາມາດເອົາເອກະສານທັ້ງໝົດໃຫ້ AI ຕອບໄດ້ ເນື່ອງຈາກບາງເອກະສານມີຫລາຍຫນ້າ
        Tokenizer_model: ແມ່ນ Model ທີ່ເຮົາຈະໃຊ້ໃນການເຮັດ chunking ຂໍ້ມູນ ເພື່ອການຄົ້ນຫາຂໍ້ມູນດ້ວຍ ChromaDB
        Max_token_limit: ແມ່ນການແບ່ງສັດສ່ວນໃຫ້ເຫມາະສົມກັບ chunk_size
        
        Args:
            docs: List of LangChain Document objects
            chunk_size: Target size for each chunk in tokens
            chunk_overlap: Number of overlapping tokens between chunks
            tokenizer_model: Path to tokenizer model
            max_token_limit: Maximum tokens allowed
            
        Returns:
            List of chunked LangChain Document objects
        """
        
        if not docs:
            print("⚠️  No documents provided for chunking")
            return []
        
        # Load tokenizer
        try:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
            print(f"✅ Loaded tokenizer: {tokenizer_model}")
        except Exception as e:
            print(f"❌ Error loading tokenizer: {e}") 
        
        # Validate parameters
        if chunk_size >= max_token_limit:
            chunk_size = max_token_limit - 500  # Safe buffer
            print(f"⚠️  Adjusted chunk_size to {chunk_size} for safety")
        
        if chunk_overlap >= chunk_size:
            chunk_overlap = chunk_size // 5  # 20% overlap
            print(f"⚠️  Adjusted chunk_overlap to {chunk_overlap}")
        
        # Create tokenizer-aware text splitter
        text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer=tokenizer,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            strip_whitespace=True,
            separators=[
                "\n\n",      # Paragraph breaks
                "\n",        # Line breaks
                ". ",        # Sentence endings
                "! ",        # Exclamation endings  
                "? ",        # Question endings
                "; ",        # Semicolon breaks
                ", ",        # Comma breaks
                " ",         # Word breaks
                ""           # Character level
            ]
        )
        
        # Split documents
        print(f"🔄 Chunking {len(docs)} documents...")
        chunked_docs = text_splitter.split_documents(docs)
        
        # Validate token counts and add metadata
        validated_chunks = []
        max_tokens_found = 0
        
        for i, chunk in enumerate(chunked_docs):
            # Count actual tokens
            token_count = len(tokenizer.encode(chunk.page_content))
            max_tokens_found = max(max_tokens_found, token_count)
            
            # Add chunk metadata
            if chunk.metadata is None:
                chunk.metadata = {}
                
            chunk.metadata.update({
                'chunk_id': i,
                'token_count': token_count,
                'char_count': len(chunk.page_content),
                'chunk_method': 'tokenizer_based'
            })
            
            # Skip if too large
            if token_count > max_token_limit:
                print(f"⚠️  Skipping oversized chunk {i}: {token_count} tokens")
                continue
                
            validated_chunks.append(chunk)
        
        # Print summary
        print(f"✅ Created {len(validated_chunks)} chunks")
        print(f"📊 Max tokens in any chunk: {max_tokens_found}")
        
        return validated_chunks
    
    @staticmethod
    def create_vector_store(
        chunked_docs: List[Document],
        embedding_model: str = "D:/model/BAAI-bge-m3",
        collection_name: str = "pdf_documents",
        persist_directory: str = "./chroma_db",
        batch_size: int = 50
    ) -> Chroma:
        """
        ສ້າງ Vector Store ດ້ວຍ ChromaDB ຈາກ chunked documents
        
        Embedding_model: ແມ່ນ Model ທີ່ໃຊ້ໃນການເຮັດ Embedding ເພື່ອປ່ຽນຂໍ້ຄວາມເປັນ Vector
        Collection_name: ແມ່ນຊື່ຂອງ Collection ໃນ ChromaDB ສາມາດສ້າງຕາມໃຈ ທີ່ຕ້ອງການ ແນະນຳໃຫ້ສ້າງເປັນ Folder ຂອງໃຜມັນ ແລະ point ໄປ Folder ນັ່ນ ເພາະວ່າ ChromaDB ເວລາມັນບັນທືກມັນຈະບັນທືກ unique key ເຊີ່ງມັນຈະເຮັດໃຫ້ເຮົາຈຳແນກຢາກ
        Persist_directory: ແມ່ນໂຟລເດີຢຸ້ບັນທຶກ ChromaDB
        Batch_size: ແມ່ນຈຳນວນ chunks ທີ່ເຮັດ embedding ຕໍ່ຄັ້ງ ເພື່ອປ້ອງກັນ memory overflow
        
        Args:
            chunked_docs: List of chunked Document objects
            embedding_model: Path to embedding model
            collection_name: Name for ChromaDB collection
            persist_directory: Directory to save ChromaDB
            batch_size: Number of documents to process at once
            
        Returns:
            Chroma vector store object
        """
        
        if not chunked_docs:
            print("⚠️  No chunked documents provided")
            return None
        
        # Create embeddings
        try:
            print(f"🔄 Loading embedding model: {embedding_model}")
            embeddings = HuggingFaceEmbeddings(
                model_name=embedding_model,
                model_kwargs={'device': 'cpu'},  # ປ່ຽນເປັນ 'cuda' ຖ້າມີ GPU
                encode_kwargs={'normalize_embeddings': True}
            )
            print(f"✅ Loaded embedding model successfully")
        except Exception as e:
            print(f"❌ Error loading embedding model: {e}")
            return None
        
        # Create ChromaDB client and collection
        try:
            # ສ້າງໂຟລເດີຖ້າຍັງບໍ່ມີ
            os.makedirs(persist_directory, exist_ok=True)
            
            print(f"🔄 Creating ChromaDB collection: {collection_name}")
            
            # ລືບ collection ເກົ່າຖ້າມີ (ປ້ອງກັນຂໍ້ຜິດພາດ)
            try:
                client = chromadb.PersistentClient(path=persist_directory)
                try:
                    client.delete_collection(collection_name)
                    print(f"🗑️  Deleted existing collection: {collection_name}")
                except:
                    pass  # Collection ບໍ່ມີຢູ່ແລ້ວ
            except Exception as e:
                print(f"⚠️  Warning during cleanup: {e}")
            
            # ສ້າງ vector store ແບບ batch
            print(f"🔄 Processing {len(chunked_docs)} documents in batches of {batch_size}")
            
            vector_store = None
            total_processed = 0
            
            for i in range(0, len(chunked_docs), batch_size):
                batch = chunked_docs[i:i + batch_size]
                batch_num = (i // batch_size) + 1
                total_batches = (len(chunked_docs) + batch_size - 1) // batch_size
                
                print(f"📦 Processing batch {batch_num}/{total_batches} ({len(batch)} documents)")
                
                try:
                    if vector_store is None:
                        # ສ້າງ vector store ທຳອິດ
                        vector_store = Chroma.from_documents(
                            documents=batch,
                            embedding=embeddings,
                            collection_name=collection_name,
                            persist_directory=persist_directory
                        )
                    else:
                        # ເພີ່ມ documents ໃໝ່ເຂົ້າໄປ
                        vector_store.add_documents(batch)
                    
                    total_processed += len(batch)
                    print(f"✅ Batch {batch_num} completed. Total processed: {total_processed}")
                    
                except Exception as e:
                    print(f"❌ Error processing batch {batch_num}: {e}")
                    continue
            
            # ບັນທຶກການປ່ຽນແປງ
            if vector_store:
                vector_store.persist()
                print(f"💾 Vector store saved to: {persist_directory}")
                
                collection_count = vector_store._collection.count()
                print(f"📊 Total vectors in collection: {collection_count}")
                print(f"📚 Collection name: {collection_name}")
                
                return vector_store
            else:
                print("❌ Failed to create vector store")
                return None
                
        except Exception as e:
            print(f"❌ Error creating vector store: {e}")
            return None
        
    @staticmethod
    def load_existing_vector_store(
        embedding_model: str = "D:/model/BAAI-bge-m3",
        collection_name: str = "pdf_documents", 
        persist_directory: str = "./chroma_db"
    ) -> Optional[Chroma]:
        """
        ໂຫຼດ Vector Store ທີ່ມີຢູ່ແລ້ວຈາກ ChromaDB
        
        Args:
            embedding_model: Path to embedding model
            collection_name: Name of ChromaDB collection
            persist_directory: Directory where ChromaDB is saved
            
        Returns:
            Chroma vector store object or None
        """
        
        try:
            # ກວດສອບວ່າມີໂຟລເດີຫຼືບໍ່
            if not os.path.exists(persist_directory):
                print(f"❌ Directory not found: {persist_directory}")
                return None
            
            # ໂຫຼດ embedding model
            embeddings = HuggingFaceEmbeddings(
                model_name=embedding_model,
                model_kwargs={'device': 'cpu'},
                encode_kwargs={'normalize_embeddings': True}
            )
            
            # ໂຫຼດ vector store
            vector_store = Chroma(
                collection_name=collection_name,
                embedding_function=embeddings,
                persist_directory=persist_directory
            )
            
            # ກວດສອບວ່າມີຂໍ້ມູນຫຼືບໍ່
            collection_count = vector_store._collection.count()
            if collection_count > 0:
                print(f"✅ Loaded existing vector store: {collection_name}")
                print(f"📊 Total vectors: {collection_count}")
                return vector_store
            else:
                print(f"⚠️  Collection '{collection_name}' is empty")
                return None
                
        except Exception as e:
            print(f"❌ Error loading vector store: {e}")
            return None
        
    @staticmethod
    def search_similar_documents(
        vector_store: Chroma,
        query: str,
        k: int = 5
    ) -> List[tuple]:
        """
        ຄົ້ນຫາເອກະສານທີ່ຄ້າຍຄືກັນ
        vector_store: ແມ່ນຂໍ້ມູເຮົາເຄີຍສ້າງ Vector Store ໃນ ./chroma_db
        query: ຄຳຖາມທີ່ຕ້ອງການຄົ້ນຫາ
        k: ຈຳນວນຜົນລັບທີ່ຕ້ອງການ
            
        Returns:
            List of tuples (document, score)
        """
        
        try:
            print(f"🔍 Searching for: {query}")
            
            # ຄົ້ນຫາດ້ວຍ score
            results = vector_store.similarity_search_with_score(
                query=query,
                k=k
            )
            
            # ສະແດງຜົນລັບ
            # for i, (doc, score) in enumerate(results):
            #     similarity = 1 - score  # ປ່ຽນ distance ເປັນ similarity
            #     print(f"\n📄 Result {i+1} (Similarity: {similarity:.3f}):")
            #     print(f"   📁 Source: {doc.metadata.get('source_file', 'Unknown')}")
            #     print(f"   📄 Page: {doc.metadata.get('page', 'Unknown')}")
            #     print(f"   🔖 Chunk: {doc.metadata.get('chunk_id', 'Unknown')}")
            #     print(f"   📝 Content preview: {doc.page_content[:100]}...")
            
            return results
            
        except Exception as e:
            print(f"❌ Error during search: {e}")
            return []

In [3]:
class GroqRAGSystem:
    """
    ລະບົບ RAG ປະສົມກັບ Groq LLM ເພື່ອຕອບຄຳຖາມອ້າງອີງຈາກເອກະສານ
    """
    
    def __init__(self, groq_api_key: str, model_name: str = "openai/gpt-oss-120b"):
        """
        ເລີ່ມຕົ້ນ GroqRAGSystem
        
        Args:
            groq_api_key: Groq API key (ຕ້ອງໄປສະໝັກທີ່ https://console.groq.com)
            model_name: ຊື່ Model ທີ່ຈະໃຊ້ (ຍົກຕົວຢ່າງ: openai/gpt-oss-120b)
        """
        self.client = Groq(api_key=groq_api_key)
        self.model_name = model_name
        
    def create_context_from_documents(self, search_results: List[tuple]) -> str:
        """
        ສ້າງ context ຈາກຜົນການຄົ້ນຫາເອກະສານ
        
        Args:
            search_results: List of tuples (document, score) ຈາກ vector search
            
        Returns:
            ຂໍ້ຄວາມ context ສຳລັບ LLM
        """
        if not search_results:
            return "ບໍ່ພົບເອກະສານທີ່ກ່ຽວຂ້ອງ"
            
        context_parts = []
        for i, (doc, score) in enumerate(search_results):
            similarity = 1 - score
            source_info = f"ແຫຼ່ງ: {doc.metadata.get('source_file', 'Unknown')} (ໜ້າ {doc.metadata.get('page', 'Unknown')})"
            content = doc.page_content.strip()
            
            context_parts.append(f"ເອກະສານ {i+1} (ຄວາມຄ້າຍຄື: {similarity:.3f}):\n{source_info}\n{content}\n")
            
        return "\n---\n".join(context_parts)
    
    def generate_answer(self, query: str, context: str) -> str:
        """
        ສ້າງຄຳຕອບໂດຍໃຊ້ Groq LLM ພ້ອມ context ຈາກເອກະສານ
        
        Args:
            query: ຄຳຖາມຂອງຜູ້ໃຊ້
            context: Context ຈາກເອກະສານ
            
        Returns:
            ຄຳຕອບຈາກ LLM
        """
        
        # ສ້າງ prompt ສຳລັບ RAG
        prompt = f"""ທ່ານເປັນ AI Assistant ທີ່ຊ່ຽວຊານໃນການຕອບຄຳຖາມໂດຍອ້າງອີງຈາກເອກະສານທີ່ໃຫ້ມາ.

ຄຳແນະນຳ:
1. ຕອບຄຳຖາມໂດຍອ້າງອີງຈາກເອກະສານທີ່ໃຫ້ມາເທົ່ານັ້ນ
2. ຖ້າບໍ່ພົບຄຳຕອບໃນເອກະສານ, ໃຫ້ບອກວ່າບໍ່ພົບຂໍ້ມູນທີ່ກ່ຽວຂ້ອງ
3. ລະບຸແຫຼ່ງຂໍ້ມູນທີ່ໃຊ້ໃນການຕອບ
4. ຕອບເປັນພາສາລາວ ແລະ ໃຫ້ຄຳຕອບທີ່ຊັດເຈນ, ລະອຽດ
5. ຕອບໃຫ້ເປັນ Format markdown

ເອກະສານອ້າງອີງ:
{context}

ຄຳຖາມ: {query}

ຄຳຕອບ:"""

        try:
            # ສົ່ງ request ໄປ Groq
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model=self.model_name,
                temperature=0.1,  # ຄວາມສ້າງສັນຕ່ຳ ເພື່ອຄວາມແມ່ນຍຳ  ຂຶ້ນນຳ Model ເພາະຄ່າ temperature ແຕ່ລະເຈົ້າມັນຕ່າງກັນ
                max_tokens=2000,  # ຈຳນວນ tokens ສູງສຸດ 
            )
            
            answer = chat_completion.choices[0].message.content
            return answer
            
        except Exception as e:
            return f"❌ ເກີດຂໍ້ຜິດພາດໃນການສ້າງຄຳຕອບ: {str(e)}"
    
    def query_documents(self, vector_store: Chroma, query: str, k: int = 5) -> dict:
        """
        ຄຳຖາມແບບສົມບູນຈາກການຄົ້ນຫາເອກະສານຈົນເຖີງການສ້າງຄຳຕອບ
        
        Args:
            vector_store: ChromaDB vector store
            query: ຄຳຖາມຂອງຜູ້ໃຊ້
            k: ຈຳນວນເອກະສານທີ່ຈະຄົ້ນຫາ
            
        Returns:
            dict ທີ່ປະກອບດ້ວຍ answer, context, ແລະ sources
        """
        
        print(f"\n🤖 Processing query: {query}")
        
        # 1. ຄົ້ນຫາເອກະສານທີ່ກ່ຽວຂ້ອງ
        search_results = DocumentLoader.search_similar_documents(
            vector_store=vector_store,
            query=query,
            k=k
        )
        
        if not search_results:
            return {
                "answer": "❌ ບໍ່ພົບເອກະສານທີ່ກ່ຽວຂ້ອງກັບຄຳຖາມຂອງທ່ານ",
                "context": "",
                "sources": []
            }
        
        # 2. ສ້າງ context ຈາກຜົນການຄົ້ນຫາ
        context = self.create_context_from_documents(search_results)
        
        # 3. ສ້າງຄຳຕອບດ້ວຍ LLM
        print("🧠 Generating answer with Groq LLM...")
        answer = self.generate_answer(query, context)
        
        # 4. ສ້າງລາຍຊື່ແຫຼ່ງຂໍ້ມູນ
        sources = []
        for doc, score in search_results:
            similarity = 1 - score
            sources.append({
                "source_file": doc.metadata.get('source_file', 'Unknown'),
                "page": doc.metadata.get('page', 'Unknown'),
                "similarity": f"{similarity:.3f}",
                "content_preview": doc.page_content
            })
        
        return {
            "answer": answer,
            "context": context,
            "sources": sources
        }

In [4]:
def main():
    """
    ຟັງຊັ່ນຫຼັກສຳລັບການທົດສອບລະບົບ RAG ກັບ Groq
    """
    
    # ການຕັ້ງຄ່າ
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # ແທນຄ່າດ້ວຍ API key ຈິງ
    
    # ລາຍຊື່ໄຟລ໌ PDF (ຖ້າຕ້ອງການສ້າງ vector store ໃໝ່)
    pdf_files = [ 
        "C:/Users/Dell/Desktop/FINAL_2024.pdf"
    ]
    
    # ກວດສອບວ່າມີ vector store ຢູ່ແລ້ວຫຼືບໍ່
    
    display(Markdown("## 🔍 ກວດສອບ Vector Store")) 
    loaded_vectorstore = DocumentLoader.load_existing_vector_store(
        embedding_model="D:/model/BAAI-bge-m3", 
        collection_name="pdf_documents", 
        persist_directory="./chroma_db"
    )
    
    # ຖ້າບໍ່ມີ vector store, ສ້າງໃໝ່
    if loaded_vectorstore is None:
        display(Markdown("## 📚 Creating new vector store..."))  
        
        # 1. ໂຫຼດເອກະສານ
        documents = DocumentLoader.load_docs(pdf_files) 
        
        if not documents:
            print("❌ No documents found. Please check your PDF file paths.")
            return
            
        # 2. ເຮັດ chunking ຂໍ້ມູນ 
        display(Markdown("## ✂️ Chunking documents...")) 
        # ເຮັດ chunking ຂໍ້ມູນ
        # ໃຊ້ Model ຂອງ BAAI-bge-m3 ເພື່ອຮັບຄ່າການເຮັດ chunking ຂໍ້ມູນ ເຊີ່ງຜູ້ໃຊ້ແມ່ນສາມາດເລືອກໄດ້ຕາມໃຈເລີຍວ່າຈະ ໃຊ້ Model ຍັງໃນການເຮັດ Embedding ສາມາດໂຫລດຜ່ານ Hugginface ໄດ້ ໂດຍກຳນົດ path ເອງ ສາມາດ ເຂົ້າໄປໃນ Folder Download Model/download-model.ipynb ເພື່ອດາວໂຫລດ Model ຍັງ
        # ກຳນັດຄ່າຕ່າງໆຂອງ chunking ໂດຍ Base on ຈາກເອກະສານ ຖ້າ ມີເອກະສານຫລາຍຫນ້າ ແນະນຳໃຫ້ລອງເພິ່ມຄ່າ chunk_size ແລະ chunk_overlap ເພື່ອຮັບຄ່າທີ່ດີກວ່າ
        chunk_documents = DocumentLoader.chunk_documents_standard(
            documents, 
            chunk_size=4000, 
            chunk_overlap=400, 
            tokenizer_model="D:/model/BAAI-bge-m3", 
            max_token_limit=8000
        )
        
        if not chunk_documents:
            print("❌ Failed to chunk documents.")
            return
            
        # 3. ສ້າງ vector store 
        display(Markdown("## 🔄 Creating vector store...")) 
        # ເຮັດ Embedding ຂໍ້ມູນ
        # ກໍລະນີນີ້ຈະຖ້າດົນແນ່ ເນື່ອງຈາກວ່າ ຈະມີການເອົາ ເອກະສານທີ່ເຮົາ Chunking ມາແປງເປັນ Vector ເພື່ອບັນທືກໃນ ChromaDB ຖ້າຢາກໃຫ້ໄວ້ ໃຜມີ GPU ແນະນຳໃຫ້ໃຊ້ cuda ແທນ cpu
        loaded_vectorstore = DocumentLoader.create_vector_store(chunk_documents)
        
        if loaded_vectorstore is None:
            print("❌ Failed to create vector store.")
            return
    
    # ເລີ່ມຕົ້ນລະບົບ RAG ກັບ Groq 
    display(Markdown("## 🚀 Initializing Groq RAG System...")) 
    
    if GROQ_API_KEY == "ໃສ່ Groq API Key ຂອງເຈົ້າທີ່ນີ້":
        print("❌ ກະລຸນາໃສ່ Groq API Key ຂອງເຈົ້າໃນຕົວແປ GROQ_API_KEY")
        print("💡 ສາມາດໄດ້ API key ຟຣີທີ່: https://console.groq.com")
        return
    
    try:
        rag_system = GroqRAGSystem(
            groq_api_key=GROQ_API_KEY,
            model_name="meta-llama/llama-4-maverick-17b-128e-instruct" 
        )
        display(Markdown("## ✅ Groq RAG System initialized successfully !"))  
        
    except Exception as e:
        print(f"❌ Error initializing Groq system: {e}")
        return
    
    # ທົດສອບລະບົບດ້ວຍຄຳຖາມຕົວຢ່າງ
    test_queries = [
        "RAG ກັບ Fine-tuning ມີຄວາມແຕກຕ່າງກັນແນວໃດ ?"
    ]
    
    display(Markdown("## 🧪 ການທົດສອບລະບົບ RAG"))
    display(Markdown("ທົດສອບດ້ວຍຄຳຖາມຕົວຢ່າງ 4 ຄຳຖາມ"))
    display(Markdown("---"))
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n📝 ຄຳຖາມທີ່ {i}: {query}")
        print("-" * 40)
        display(Markdown(f"### 📝 ຄຳຖາມທີ່ {i}: {query}"))
        display(Markdown("---"))
        
        # ສົ່ງຄຳຖາມໄປລະບົບ RAG
        result = rag_system.query_documents(
            vector_store=loaded_vectorstore,
            query=query,
            k=5  # ຄົ້ນຫາ 5 ເອກະສານທີ່ກ່ຽວຂ້ອງ
        )
        
        # ສະແດງຜົນລັບ
        display(Markdown("#### 🤖 ຄຳຕອບ:"))
        display(Markdown(f"""
            ```
            {result['answer']}
            ```
        """))
        
        if result['sources']:
            display(Markdown("#### 📚 ແຫຼ່ງຂໍ້ມູນອ້າງອີງ:"))
                
            sources_md = ""
            for j, source in enumerate(result['sources'], 1):
                    sources_md += f"""
                **{j}.** `{source['source_file']}` (ໜ້າ {source['page']}) - ຄວາມຄ້າຍຄື: `{source['similarity']}`
                > {source['content_preview']}...

            """
            display(Markdown(sources_md))
                    
        display(Markdown("---"))
    
    # ໂໝດ interactive ສຳລັບຜູ້ໃຊ້ສາມາດຖາມຄຳຖາມເອງ
    display(Markdown("## 💬 ໂໝດ Interactive - ພິມຄຳຖາມຂອງທ່ານ (ພິມ 'quit' ເພື່ອອອກ")) 
    display(Markdown("---"))
    
    while True:
        try:
            user_query = input("\n❓ ຄຳຖາມຂອງທ່ານ: ").strip()
            
            if user_query.lower() in ['quit', 'exit', 'ອອກ']:
                print("👋 ຂອບໃຈທີ່ໃຊ້ລະບົບ RAG!")
                break
                
            if not user_query:
                print("⚠️ ກະລຸນາໃສ່ຄຳຖາມ")
                continue
            
            display(Markdown(f"### ❓ ຄຳຖາມ: `{user_query}`"))
            
            # ສົ່ງຄຳຖາມໄປລະບົບ RAG
            result = rag_system.query_documents(
                vector_store=loaded_vectorstore,
                query=user_query,
                k=5
            )
            
            # ສະແດງຜົນລັບ
            display(Markdown("#### 🤖 ຄຳຕອບ:"))
            display(Markdown(f"""
                ```
                {result['answer']}
                ```
            """))
            
            # ສະແດງແຫຼ່ງຂໍ້ມູນ (ແບບຫຍໍ້)
            if result['sources']: 
                display(Markdown("#### 📚 ແຫຼ່ງຂໍ້ມູນອ້າງອີງ:"))
                for source in result['sources'][:3]:  # ສະແດງ 3 ແຫຼ່ງທຳອິດ
                    display(Markdown(f"#### • {source['source_file']} (ໜ້າ {source['page']})")) 
            
        except KeyboardInterrupt:
            print("\n\n👋 ຂອບໃຈທີ່ໃຊ້ລະບົບ RAG!")
            break
        except Exception as e:
            print(f"❌ ເກີດຂໍ້ຜິດພາດ: {e}")

In [5]:
if __name__ == "__main__":
    main()

## 🔍 ກວດສອບ Vector Store

❌ Directory not found: ./chroma_db


## 📚 Creating new vector store...

✅ Processed PDF: C:/Users/Dell/Desktop/FINAL_2024.pdf (95 pages)
📚 Total PDF documents loaded: 95


## ✂️ Chunking documents...

...


                **1.** `FINAL_2024.pdf` (ໜ້າ 10) - ຄວາມຄ້າຍຄື: `-0.230`
                > ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ�າທີ 7 
     
  
   
 
 
My QR 
 ໃຊ�ສ�າລັບສ�າງ QRຂອງເລກບັນຊີ  ແທນການສົ່ງເລກບັນຊີ 
ໃນການໂອນເງິນໃຫ�ກັນ. ໂດຍສະແກນຜ�ານ OnePay ກ�
ຈະສະແດງບັນຊີປາຍທາງໃຫ�ທ�ານ ເພີ່ມຄວາມສະດວກ 
ແລະ ວ�ອງໄວກວ�າ). 
 
          ອອກບັດຊິບ 
 ເປັນຟັງຊັນອອກບັດຊິບ Chip Card) ກັບ BCEL One 
ແບບອອນລາຍໂດຍບ�່ຕ�ອງເຂົ້າມາ Counter. 
 
              OneX 
 ເປັນຟັງຊັນ ແພລດຟອມການຄ�າອິເລັກໂທຼນີດ ທີ່ມີການຊື້ -
ຂາຍສິນຄ�າ ແລະ ການບ�ລິການ ແລະສາມາດຂົນສົ່ງຈາກ
ຫຼາຍບ�ລິສັດໄດ�. 
 
ສັ່ງຈອງຊື້ພັນທະບັດ 
 ເປັນຟັງຊັນໃນການຈອງແລະຊື້ພັນທະບັດຂອງລັດຖະບານ 
ຕາມທີ່ມີການແຈ�ງການປະກາດຂາຍໃນແຕ�ລະໄລຍະ ໂດຍ)
ການຊື້ແມ�ນເມື່ອຮອດເວລາກ�ານົດ ຈະມີການຈ�າຍດອກ
ເບ�ຍ ແລະເງິນລົງທຶນໃຫ�ຕາມຈ�ານວນ%ປະກາດຂາຍ . 
 
               ຕະຫຼາດແລກປ�ຽນ 
 ເປັນຟັງຊັນລະບົບແລກປ�ຽນເງິນຕາລະຫວ�າງລູກ�າ ແລະ ລູກ
ຄ�າຜູ�ໃຊ�ບ�ລິການສາມາດກ�ານົດອັດຕາແລກປ�ຽນ ແລະ  ,
ເລືອກວົງເງິນບົນພື້ນຖານຂອບເຂດທີ່ລະບົບໄດ�ກ�ານົດໄວ�  
 ຖ�າຫາກວ�າການແລກປ�ຽນເງິນຕາຈັບຄູ�ກັນໄດ� ຫຼື ລະບົບຈັບ
ຄູ�ໃຫ�ຕາມຄ�າສັ່ງຜູ�ໃຊ�ບ�ລິການ ລະບົບຈະຕັດເງິນທັນທີ. 
 TAP ຕັ້ງຄ�າ 
 ປ�ຽນລະຫັດຜ�ານ  ປ�ຽນລະຫັດຜ�ານທີ່ໃຊ�ເຂົ້າສູ�ລະບົບ. 
ປ�ຽນ 3 ຄ�າຖາມລັບ 
 ເປັນຟັງຊັນໃຫ�ລູກທີ່ຕ�ອງການປ�ຽນ3 ຄ�າຖາມລັບ. 
 ຕັ້ງຄ�າລາຍນີ້ວມື 
 ເປັນການຕັ້ງຄ�າເພີ່ມລາຍນິ້ວມື ສາມາດສະແກນລາຍນິ້ມມື
ແທນການພິມລະຫັດຜ�ານຕ�າງໆ ເພື່ອ ເປີດ/ປິດ ການເຂົ້າສູ�
ລະບົບ ແລະ ການເພີ່ມເລກບັນຊີເພື່ອໂອນເງິນດ�ວຍລາຍນິ້ວ
ມື. 
 ລັອກ/ປົດລັອກບັດ 
 ສາມາດໃຊ� ເປີດ/ປິດ ການນ�າໃຊ�ບັດ ໃນກ�ລະນີບັດເສຍ, ບັດ
ຖືກລັກ ຫຼື ຕ�ອງການລະງ�ບບັດໄວ�ຊົ່ວຄາວ. ຖ�າບັດຖືກລັອກ 
ຈະບ�່ສາມາດນ�າໃຊ�ບັດເພື່ອຖອນເງິນ ແລະ ຊ�າລະຄ�າບ�ລິການ
ຕ�າງໆຜ�ານ BCEL One ໄດ�....

            
                **2.** `FINAL_2024.pdf` (ໜ້າ 81) - ຄວາມຄ້າຍຄື: `-0.236`
                > ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ�າທີ 78 
 ໄດ�ຮັບຄະແນນສະສົມໄມລ�ເພີ່ມຕື່ມ 3% ຈາກຈ�ານວນຄະແນນທີ່ໄດ�ໃນຖ�ຽວບິນນັ້ນ ຫຼື ຄັ້ງນັ້ນຂອງທຸກໆ
ເສັ້ນທາງບິນ ທັງເສັ້ນທາງພາຍໃນ ແລະ ຕ�າງປະເທດ. 
ສ�າຄັນ  :ກ�ລະນີທີ່ທ�ານມີທັງບັດສະມາຊິກຂອງການບິນລາວ ແລະ ບັດຮ�ວມພິເສດ Co- brand, ທ�ານສາມາດ
ເລືອກນ�າໃຊ�ສິດທິພິເສດຈາກບັດໃດບັດໜຶ່ງເທົ່ານັ້ນ  .ບ�່ສາມາດສະເໜີທັງສອງບັດ ເພື່ອຮັບສິດທິພິເສດເປັນ 2 
ເທົ່າ.  
ບັດ Co- brand 
Gold Sky 
 ໃຊ�ເປັນບັດສະມາຊິກສະສົມໄມລ�ເພື່ອແລກຂອງສົມມະນາຄຸນຈາກການບິນລາວ, 
 ໄດ�ຮັບສ�ວນຫ�ດ 30%  ສ�າລັບຫ�ອງພັກຮັບຮອງພິເສດຈາກການບິນລາວ ສະເພາະຢູ�ສະ ໜາມບິນປາຍທາງທີ່
ການບິນລາວມີ ແລະ ແຕ�ຜູ�ຖືບັດຕ�ອງແຈ�ງພະນັກງານການບິນລາວ ທີ່ຈຸດ Check- in ຖ�າທ�ານຕ�ອງການນ�າ
ໃຊ�ຫ�ອງພັກຮັບຮອງພິເສດຈາກການບິນລາວ). 
 ໃຊ�ສະສົມໄມລ� ເພື່ອແລກຊື້ປີ້ເຮືອບິນກັບການບິນລາວ ແລະ ຍົກລະດັບຈາກປີ້ທ�ໍາມະດາ ເປັນປີ້ບ�ອນນັ່ງ
ທຸລະກິດ ແຕ�ຕ�ອງແຈ�ງໃຫ�ພະນັກງານ ການບິນລາວຮັບຮູ� 24 ຊົ່ວໂມງກ�ອນການເດີນທາງ). 
 ໃຊ�ສະສົມໄມລ� ເພື່ອແລກຊື້ປີ້ເຮືອບິນກັບການບິນໃຫ�ແກ�ຕົນເອງ, ຄອບຄົວ ຫຼື ໝູ�ເພື່ອນ. 
 ໄດ�ຮັບຄະແນນສະສົມໄມລ�ເພີ່ມຕື່ມ 2% ຈາກຈ�ານວນຄະແນນທີ່ໄດ�ໃນຖ�ຽວບິນນັ້ນ ຫຼື ຄັ້ງນັ້ນຂອງທຸກໆ
ເສັ້ນທາງບິນ ທັງເສັ້ນທາງພາຍໃນ ແລະ ຕ�າງປະເທດ. 
 ໄດ�ຮັບສິດອັນດັບທີ 2 ໃນການໄດ�ບ�ອນກ�ອນ ກ�ລະນີລ�ຖ�າ Stand by list 
 ໄດ�ຮັບສ�ວນຫ�ດເມື່ອເຂົ້າພັກທີ່ໂຮງແຮມເມືອງທອງ ແຂວງ ຫຼວງພະບາງ ອີງຕາມໂປຼໂມຊັ້ນທີ່ໂຮງແຮມ
ອອກແຕ�ລະໄລຍະ 
 ສາມາດແຈ�ງປີ້ເຮືອບິນທີ່ປ�ອງບ�ລິການແຈ�ງປີ້ຂອງຊັ້ນທຸລະກິດໄດ� Business Check- in Counter) 
 ໄດ�ຮັບນະໂຍບາຍນ�້າໜັກເກີນ 8 Kg, ສິດນີ້ບ�່ສາມາດໃຊ�ໄດ�ກັບສາຍການບິນອື່ນ Code Share Flight) 
 ໄດ�ຮັບສິດພິເສດອັນດັບທີ່ 2 ໃນການຂຶ້ນເຮືອບິນກ�ອນເວລາ Boarding 
 ສາມາດເຂົ້າໃຊ�ບ�ລິການທີ່ປ�ອງຈອງ-ຂາຍປີ້ການບິນລາວໂດຍບ�່ຕ�ອງກົດບັດຄິວ 
 ສ�າຄັນ:  ກ�ລະນີທີ່ທ�ານມີທັງບັດສະມາຊິກຂອງການບິນລາວ ແລະ ບັດຮ�ວມພິເສດ Co- brand, ທ�ານ
ສາມາດເລືອກນ�າໃຊ�ສິດທິພິເສດຈາກບັດໃດບັດໜຶ່ງເທົ່ານັ້ນ. ບ�່ສາມາດສະເໜີທັງສອງບັດ ເພື່ອຮັບສິດທິ
ພິເສດເປັນ 2 ເທົ່າ.   
 
 
ບັດ Co- brand 
Silver Cloud 
 ໃຊ�ເປັນບັດສະມາຊິກສະສົມໄມລ�ເພື່ອແລກຂອງສົມມະນາຄຸນຈາກການບິນລາວ 
 ໄດ�ຮັບສ�ວນຫ�ດ 20%  ສ�າລັບຫ�ອງພັກຮັບຮອງພິເສດຈາກການບິນລາວ ສະເພາະຢູ�ສະໝາມບິນໝາຍ
ປາຍທາງທີ່ການບິນລາວມີ ແລະ ແຕ�ຜູ�ຖືບັດຕ�ອງແຈ�ງພະນັກງານການບິນລາວ ທີ່ຈຸດ Check- in ຖ�າທ�ານ
ຕ�ອງການນ�າໃຊ�ຫ�ອງພັກຮັບຮອງພິເສດຈາກການບິນລາວ). 
 ໃຊ�ສະສົມໄມລ� ເພື່ອແລກຊື້ປີ້ເຮືອບິນກັບການບິນໃຫ�ແກ�ຕົນເອງ, ຄອຍຄົວ, ຫຼື ໝູ�ເພື່ອນ. 
 ໄດ�ຮັບສິດອັນດັບທີ 3 ໃນການໄດ�ບ�ອນກ�ອນ ກ�ລະນີລ�ຖ�າ Stand by list 
 ໄດ�ຮັບຄະແນນສະສົມໄມລ�ເພີ່ມຕື່ມ 1% ຈາກຈ�ານວນຄະແນນທີ່ໄດ�ໃນຖ�ຽວບິນນັ້ນ ຫຼື ຄັ້ງນັ້ນຂອງທຸກໆ
ເສັ້ນທາງບິນ ທັງເສັ້ນທາງພາຍໃນ ແລະ ຕ�າງປະເທດ. 
 ໄດ�ຮັບສ�ວນຫ�ດເມື່ອເຂົ້າພັກທີ່ໂຮງແຮມເມືອງທອງ ແຂວງ ຫຼວງພະບາງ ອີງຕາມໂປຼໂມຊັ້ນທີ່ໂຮງແຮມ
ອອກແຕ�ລະໄລຍະ 
 ໄດ�ຮັບນະໂຍບາຍນ�້າໜັກເກີນ 5 Kg, ສິດນີ້ບ�່ສາມາດໃຊ�ໄດ�ກັບສາຍການບິນອື່ນ Code Share Flight) 
 ສ�າຄັນ:  ກ�ລະນີທີ່ທ�ານມີທັງບັດສະມາຊິກຂອງການບິນລາວ ແລະ ບັດຮ�ວມພິເສດ Co- brand, ທ�ານ
ສາມາດເລືອກນ�າໃຊ�ສິດທິພິເສດຈາກບັດໃດບັດໜຶ່ງເທົ່ານັ້ນ. ບ�່ສາມາດສະເໜີທັງສອງບັດ ເພື່ອຮັບສິດທິ
ພິເສດເປັນ 2 ເທົ່າ. 
ສິດທິປະໂຫຍດ 1. ປະກັນໄພທ�ອງທ�ຽວໃນເວລາເດີນທາງໄປຕ�າງປະເທດ Oversea Travel Insurance) *ສ�າລັບບັດ    
Co- Brand Platinum ແລະ ບັດ Co- Brand Gold ມີປະກັນໄພ)...

            
                **3.** `FINAL_2024.pdf` (ໜ້າ 52) - ຄວາມຄ້າຍຄື: `-0.264`
                > ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ�າທີ 49 
 ທະນາຄານຈະບ�່ໃຫ�ບ�ລິການສົ່ງເງິນ  MoneyGram ໃນກ�ລະນີທີ່ຂ�້ມູນຜູ�ຮັບເງິນຢູ�ໃນລາຍຊື່ຕ�ອງຫ�າມ 
ແລະ ລະບົບ AgentConnect ບ�່ອະນຸຍາດ. 
 ຊ�ວງເວລາຮັບເງິນໂອນ ຈະຂຶ້ນຢູ�ກັບເວລາການໃຫ�ບ�ລິການຂອງຕົວແທນ MoneyGram  ທີ່ປາຍທາງ. 
 
5.5 ຜະລິດຕະພັນໂອນເງິນດ�ວນ ລະຫວ�າງປະເທດ SpeedSend 
 
ຜະລິດຕະພັນໂອນເງິນດ�ວນ ລະຫວ�າງປະເທດ SpeedSend 
ຄຸນລັກສະນະ 
ເປັນຜະລິດຕະພັນໂອນເງິນດ�ວນລະຫວ�າງປະເທດ ຫຼື ເອີ້ນວ�າ SpeedSend ທີ່ສາມາດຕອບສະໜອງ 
ແລະ ເປັນທາງເລືອກໃຫ�ແກ�ກຸ�ມລູກຄ�າຜູ�ທີ່ມີຄວາມຕ�ອງການໃນການໂອນເງິນໄປຕ�າງປະເທດ ເຊັ່ນ:  
 ແຮງງານຕ�າງປະເທດມາເຮັດວຽກໃນ ສປປ ລາວ ໂອນເງິນກັບບ�ານໃຫ�ຄອບຄົວຢູ�ຕ�າງປະເທດ ຫຼື ບັນດາ
ແຮງງານລາວທີ່ໄປເຮັດວຽກຢູ�ຕ�າງປະເທດໂອນເງິນກັບບ�ານໃຫ�ຄອບຄົວຢູ�ລາວ, 
 ເປັນອີກໜຶ່ງຊ�ອງທາງທີ່ອ�ານວຍຄວາມສະດວກໃຫ�ແກ�ພ�່ແມ�ຜູ�ປົກຄອງ ສາມາດສົ່ງເງິນໃຫ�ຄອບຄົວ ຫຼື 
ລູກຫຼານທີ່ສຶກສາຕ�່ຢູ�ຕ�າງປະເທດ.  
ຜົນປະໂຫຍດທີ່
ໄດ�ຮັບ 
 ເປັນການຫ�ດຜ�ອນຄ�າໃຊ�ຈ�າຍທີ່ຕ�ອງຈ�າຍໃຫ�ມືກາງ ຫຼື ຕະຫຼາດມືດ ທີ່ເອົາປຽບຜູ�ໂອນເງິນ. 
 ສະດວກສະບາຍດ�ານເອກະສານ ແລະ ຂັ້ນຕອນໃນການໂອນເງິນ ແລະ ຮັບເງິນ ພ�ອມດ�ວຍຄ�າທ�ານຽມ
ການບ�ລິການ ທີ່ແທດເໝາະ ແລະ ສາມາດຈ�າຍໄດ�.  
 ສາມາດໂອນ-ຮັບເງິນໄດ�ກັບປະເທດພະມ�າ. 
 ທຄຕລ ຍິນດີຊ�ວຍທ�ານໃນການແລກປ�ຽນເງິນຕາຕ�າງປະເທດໃນແຕ�ລະໄລຍະໃນການສົ່ງ-ຮັບ. 
ເງື່ອນໄຂການໃຊ�
ບ�ລິການ 
 ສ�າລັບບຸກຄົນທີ່ເປັນຄົນຕ�າງປະເທດ: ຕ�ອງມີໜັງສືເດີນທາງພ�ອມກັບວີຊ�າເຂົ້າປະເທດລາວຍັງບ�່ໝົດ
ກ�ານົດ. 
 ສ�າລັບບຸກຄົນທີ່ເປັນຄົນລາວ ແລະ ຊາວຕ�າງດ�າວ: ຕ�ອງມີບັດປະຈ�າຕົວ, ໜັງສືຜ�ານແດນ Passport) 
ຫຼື ສ�າມະໂນຄົວ ທີ່ມີອາຍຸການນ�າໃຊ�ໄດ�ຕາມກົດໝາຍ),  
  ສົ່ງ-ຮັບເງິນຢູ� ທຄຕລ ເປັນສະກຸນໂດລາສະຫະລັດ ເທົ່ານັ້ນ ການ ສົ່ງ-ຮັບເງິນແມ�ນຂຶ້ນກັບນະໂຍບາຍ, 
ລະບຽບຂອງປະເທດນັ້ນໆ)  
ຂ�້ມູນເພີ່ມຕື່ມ  ຂ�້ມູນເພີ່ມເຕີມກະລຸນາເຂົ້າທີ່ www.cimb.com.my/ en/ personal/ day- to- day-
banking/ remittance/ speedsend.html ໂທ :021 263510 
5.6 ບ�ລິການໂອນເງິນອອກ Outward Remittance ຜ�ານລະບົບ Border Trade E-Bank) 
ບ�ລິການໂອນເງິນອອກ -  Outward Remittance (ຜ�ານລະບົບ Border Trade E-Bank) 
ຄຸນລັກສະນະ 
 ທຄຕລ ສາມາດໃຫ�ບ�ລິການທ�ານທີ່ຕ�ອງການໂອນເງິນ ສະກຸນເງິນຢວນ ໄປ ສ.ປ ຈີນ ເພື່ືອຊ�າລະຄ�າສິນຄ�າ , 
ຄ�າບ�ລິການ, ຄ�າຮຽນ ແລະ ອື່ນໆ ດ�ວຍຄວາມປອດໄພ, ວ�ອງໄວ ໂດຍຜ�ານລະບົບ Border Trade E-bank 
ເຊິ່ງເປັນລະບົບສະເພາະ ການໂອນສະກຸນເງິນຢວນໄປ ສປ ຈີນ. 
ຈຸດປະສົງ ການ
ໂອນເງິນໄປຕ�າງ
ປະເທດ 
 ສາມາດຊ�າລະຄ�າສິນຄ�າ ແລະ ບ�ລິການຕ�າງໆ. 
 ສ�າລັບລູກຄ�າສ�ວນບຸກຄົນທ�ານສາມາດໂອນເງິນໄປຕ�າງປະເທດເພື່ອຊ�າລະ ຄ�າປິ່ນປົວ,  ການສຶກສາ, 
ຢ�ຽມຢາມຕ�າງປະເທດ ຕາມລະບຽບການທີ່ທະຄານແຫ�ງ ສປປລາວ ວາງອອກແຕ�ລະໄລຍະ. 
ຜົນປະໂຫຍດ 
ທີ່ໄດ�ຮັບ 
  ສາມາດໂອນເງິນໄປທຸກທະນາຄານທີ່ຢູ�ໃນ ສ.ປ ຈີນ ດ�ວຍຄວາມວອ�ງໄວ ແລະ ປອດໄພ. 
 ສາມາດໂອນອອກໄດ�ສະເພາະສະກຸນເງິນຢວນ CNY)...

            
                **4.** `FINAL_2024.pdf` (ໜ້າ 69) - ຄວາມຄ້າຍຄື: `-0.281`
                > ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ�າທີ 66 
ໝາຍເຫດ ອັດຕາດອກເບ�ຍ ແລະ ຄ�າທ�ານຽມ ອາດຈະປ�ຽນຕາມການປະກາດໃຊ�ໃນແຕ�ລະໄລຍະ 
 
9.6 ໜັງສືຄ�້າປະກັນການຈ�າຍເງິນ )Payment Guarantees) 
ໜັງສືຄ�້າປະກັນການຈ�າຍເງິນ (Payment Guarantees) 
ຄຸນລັກສະນະ  ເພື່ອຄ�້າປະກັນພັນທະການຈ�າຍເງິນຂອງຜູ�ສະເໜີ ຜູ�ຊື້). ກ�ລະນີຜູ�ສະເໜີບ�່ຊ�າລະສິນຄ�າ, ຜູ�ຮັບຜົນ
ປະໂຫຍດ ຜູ�ຂາຍ) ຈິ່ງຮຽກຮ�ອງໃຫ�ທະນາຄານຊ�າລະຄ�າສິນຄ�າແທນ. 
ຄຸນສົມບັດຂອງ
ຜູ�ສະເໜີ 
 ມີບັນຊີນ�າ ທຄຕລ 
 ໃບທະບຽນວິສາຫະກິດ  
ເອກະສານ
ປະກອບຫຼັກ 
 ສັນຍາ 
 ເອກະສານອື່ນໆຕາມແບບຟອມ ຂອງ ທຄຕລ 
ວົງເງິນໜັງສືຄ�້າ
ປະກັນ 
 ຂື້ນກັບແຕ�ລະໂຄງການ 
ອັດຕາດອກເບ�ຍ 
)ຕ�່ປີ  
 ບ�່ມີ 
ຄ�າທ�ານຽມ  ອີງຕາມການປະກາດໃຊ�ໃນແຕ�ລະໄລຍະ 
ໄລຍະເວລາ  ອີງຕາມແຕ�ລະໂຄງການ 
ຫຼັກຊັບຄ�້າປະກັນ  ບັນຊີເງິນຝາກປະຢັດ, ບັນຊີເງິນຝາກກະແສລາຍວັນ, ບັນຊີເງິນຝາກມີກ�ານົດ, ວົງເງິນສິນເຊື່ອ, 
ທະນາຄານອື່ນ. 
ໝາຍເຫດ ອັດຕາດອກເບ�ຍ ແລະ ຄ�າທ�ານຽມ ອາດຈະປ�ຽນຕາມການປະກາດໃຊ�ໃນແຕ�ລະໄລຍະ 
 
9.7 ໜັງສືຄ�້າປະກັນເພື່ອເອົາສິນຄ�າອອກຈາກທ�າເຮືອ )Trade Finance) 
ໜັງສືຄ�້າປະກັນເພື່ອເອົາສິນຄ�າອອກຈາກທ�າເຮືອ (Shipping Guarantee) 
ຄຸນລັກສະນະ  ເພື່ອເພື່ອຄ�້າປະກັນການໃຊ�ແທນເອກະສານຂົນສົ່ງຕົ້ນສະບັບ Original Bill of Lading)  
 ເພື່ອຮັບສິນຄ�າອອກຈາກທ�າເຮືອ ທັງນີ້ກ�່ເພື່ອເປັນການຫ�ດຄ�າໃຊ�ຈ�າຍໃນການເກັບຮັກສາສິນຄ�າຢູ�
ທ�າເຮືອ. 
ຄຸນສົມບັດຂອງ
ຜູ�ສະເໜີ 
 ມີບັນຊີນ�າ ທຄຕລ 
 ໃບທະບຽນວິສາຫະກິດ  
ເອກະສານ
ປະກອບຫຼັກ 
 ເອກະສານຂົນສົ່ງ Bill of Lading)  
 ເອກະສານອື່ນໆຕາມແບບຟອມ ຂອງ ທຄຕລ 
ວົງເງິນໜັງສືຄ�້າ
ປະກັນ 
 ຂື້ນກັບມູນຄ�າສິນຄ�າ 
ອັດຕາດອກເບ�ຍ 
)ຕ�່ປີ  
 ບ�່ມີ 
ຄ�າທ�ານຽມ  ອີງຕາມການປະກາດໃຊ�ໃນແຕ�ລະໄລຍະ 
ໄລຍະເວລາ  ອີງຕາມແຕ�ລະໂຄງການ 
ຫຼັກຊັບຄ�້າປະກັນ  ບັນຊີເງິນຝາກປະຢັດ, ບັນຊີເງິນຝາກກະແສລາຍວັນ, ບັນຊີເງິນຝາກມີກ�ານົດ, ວົງເງິນສິນເຊື່ອ, 
ທະນາຄານອື່ນ. 
ໝາຍເຫດ ອັດຕາດອກເບ�ຍ ແລະ ຄ�າທ�ານຽມ ອາດຈະປ�ຽນຕາມການປະກາດໃຊ�ໃນແຕ�ລະໄລຍະ...

            
                **5.** `FINAL_2024.pdf` (ໜ້າ 85) - ຄວາມຄ້າຍຄື: `-0.283`
                > ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ�າທີ 82 
 ທ�ານສາມາດຕິດຕາມລາຍການເຄື່ອນໄຫວ ແລະ ຄວມຄຸມການນ�າໃຊ�ບັດເຄຼດິດຂອງທ�ານຜ�ານ BCEL 
One, ເຊິ່ງເປັນເຄື່ອງມືທີ່ຈະຊ�ວຍໃຫ�ທ�ານນ�າໃຊ�ບັດເຄຼດິດຂອງທ�ານໄດ�ຢ�າງສະດວກສະບາຍ ແລະ ປອດ
ໄພຍິ່ງຂຶ້ນ. ສາມາດຊ�າລະຜ�ານຫຼາຍຊ�ອງທາງເຊັ່ນ: ການສັ່ງຊື້ຜ�ານເວັບໄຊ, ອີເມວ ຫຼື ໂທລະສັບ. 
ເງື່ອນໄຂ 
ການນ�າໃຊ� 
1. ບຸກຄົນສັນຊາດລາວ ຫຼື ຕ�າງປະເທດ 
2. ເປັນບຸກຄົນທີ່ມີລາຍຮັບໝັ້ນຄົງ 
3. ມີທີ່ຢູ�ແນ�ນອນ, ຖ�າເປັນຄົນຕ�າງປະເທດຕ�ອງມີໃບອະນຸຍາດ ຫຼື ການຄ�້າປະກັນຈາກອົງການທີ່ກ�ຽວຂ�ອງ 
4. ມີວົງເງິນຄ�້າປະກັນ ສ�າລັບການຄ�າດ�ວຍເງິນໂດລາແມ�ນໄດ�ວົງເງິນໃນບັດບ�່ເກີນ 85 % ຂອງວົງເງິນ   
ຄ�້າປະກັນ, ສ�ວນການຄ�້າປະກັນດ�ວຍສະກຸນອື່ນທີ່ບ�່ແມ�ນໂດລາ ແມ�ນໄດ�ວົງເງິນບ�່ເກີນ 80% ) 
5. ຖ�າບ�່ຕ�ອງການນ�າໃຊ�ວົງເງິນຄ�້າປະກັນ ແມ�ນສາມາດສະເໜີອອກບັດເຄຼດິດທີ່ບ�່ມີຫຼັກຊັບຄ�້າປະກັນ ໂດຍ
ສະເໜີຜ�ານຄະນະກ�າມະການສິນເຊື່ອຕາມລະບຽບການ 
6. ມີຄວາມສາມາດທາງດ�ານການປະພຶດຕ�່ໜ�າກົດໝາຍໃນການປະຕິບັດສັນຍາ ແລະ ເງື່ອນໄຂຕ�າງໆ 
7. ຕ�ອງມີບັນຊີກັບ ທະນາຄານການຄ�າຕ�າງປະເທດລາວ ມະຫາຊົນ ຢ�າງໜ�ອຍ 1 ບັນຊີ, ມີບັດປະຈ�າຕົວ ຫຼື 
ໜັງສືເດີນທາງ passport) ທີ່ບ�່ໝົດອາຍຸ ຫຼື ປື້ມສ�າມະໂນຄົວ 
ໝາຍເຫດ 
 ອີງຕາມລະບຽບຂອງ ທຄຕລ, ແຕ�ລະບັດມີການກ�ານົດວົງເງິນການຖອນເງິນສົດ ແລະ ການໃຊ�ຈ�າຍຕ�່ຄັ້ງ
ຕ�່ວັນ. ຕາຕະລາງກ�ຽວກັບວົງເງິນການວົງເງິນການຖອນ, ການໃຊ�ຈ�າຍ ແລະ ຄ�າແນະນ�າການນ�າໃຊ� ແມ�ນ
ລະບຸລະອຽດຢູ�ໃນໃບເງື່ອນໄຂທີ່ທາງທະນາຄານມອບໃຫ�ແກ�ທ�ານເພື່ອເປັນບ�ອນອີງໃນການນ�າໃຊ�. 
ສະນັ້ນ, ຜູ�ຖືບັດຄວນສຶກສາເງື່ອນໄຂໃຫ�ລະອຽດເພື່ອຄວາມເຂົ້າໃຈ ແລະ ສະດວກໃນການນ�າໃຊ�ບັດ 
 ແຕ�ລະປະເພດບັດ ສາມາດຖອນເງິນສົດ ຫຼື ສາມາດໃຊ�ຈ�າຍໄດ�ທັງໝົດ 100% ຂອງວົງເງິນສິນເຊື່ອ.   
ສິດທິປະໂຫຍດ 
1. ຟຣີປະກັນໄພທ�ອງທ�ຽວໃນເວລາເດີນທາງໄປຕ�າງປະເທດ Oversea Travel Insurance) 
ທຸກຄັ້້ງທີ່ທ�ານເດີນທາງໄປຕ�າງປະເທດ ແລະ ນ�າໃຊ�ບັດ BCEL World Mastercard ຂອງທ�ານຊ�າລະ
ຄ�າເດີນທາງ ປີ້ຍົນ) , ທ�ານຈະໄດ�ຮັບການຄຸ�ມຄອງຈາກແຜນປະກັນໄພທ�ອງທ�ຽວ ໃນຕະຫຼອດການ
ເດີນທາງ ເປັນຕົ້ນ:  
• ຄ�າໃຊ�ຈ�າຍປິ່ນປົວສຸກເສີນ     ສູງສຸດ 100.000 ໂດລາ 
• ຄ�າໃຊ�ຈ�າຍໃນການອົບພະຍົບ/ການຍ�າຍກັບຄືນຖິ່ນ ສູງສຸດ 1.000.000 ໂດລາ 
• ຄ�າອຸບັດຕິເຫດບຸກຄົນ-ຕະຫຼອດການເດີນທາງ ສູງສຸດ 85.000 ໂດລາ 
• ຄ�າເຂົ້າພັກຮັກສາທີ່ໂຮງໝ�    50 ໂດລາ/ວັນ ສູງສຸດເຖິງ 30 ວັນ 
• ຄ�າຊົດເຊີຍກ�ລະນີກະເປົາເດີນທາງເສຍຫາຍ  ສູງສຸດ 1.500 ໂດລາ 
• ຄ�າຊົດເຊີຍກ�ລະນີຖ�ຽວບິນລ�າຊ�າ   75 ໂດລາ/ຊົ່ວໂມງ 
• ກະເປົາເດີນທາງລ�າຊ�າ    75 ໂດລາ/ຊົ່ວໂມງ 
ແລະ ລາຍລະອຽດອື່ນໆສາມາດເບິ່ງເພີ່ມເຕີມໄດ�ຈາກເງື່ອນໄຂການຄຸ�ມຄອງປະກັນໄພ.  
• ຄຸ�ມຄອງກ�ລະນີທີ່ທ�ານນ�າໃຊ�ບັດຊ�າລະຄ�າສິນຄ�າ ແຕ�ບ�່ໄດ�ຮັບເຄື່ອງ ຫຼື ເຄື່ອງທີ່ໄດ�ຮັບບ�່ຖືກຕ�ອງ
ຕາມລາຍການທີ່ຕ�ອງການ ຫຼື ນ�າໃຊ�ບ�່ໄດ�ເນື່ອງຈາກເປ�ເພເສຍຫາຍ ສູງສຸດ 1.500 ໂດລາ. 
 
2. ການລົງທະບຽນນ�າໃຊ�ຫ�ອງພັກຮັບຮອງ LoungeKey 
BCEL World Mastercard ໄດ�ຮັບສິດເຂົ້າຫ�ອງພັກຮັບຮອງຂອງສະໜາມບິນຊັ້ນນ�າຕ�າງໆຟຣິ 2 ຄັ້ງ/
ປີ. ແຕ�ກ�ອນເຂົ້່ານ�າໃຊ�ບ�ລິການຫ�ອງພັກຮັບຮອງດັ່ງກ�າວນັ້ນ, ທ�ານຕ�ອງລົງທະບຽນຕາມຂັ້ນຕອນດັ່ງລຸ�ມນີ້:  
• ດາວໂຫຼດ Mastercard Airport Experience App ຢູ� App Store ຫຼື Play Store ແລະ 
ລົງທະບຽນຜ�ານ App. 
• ປ�ອນເລກບັດ  ແລະ ອັກສອນແລນດອມເພື່ອຢືນຢັນ: ເພື່ອການຢືນຢັນ ແລະ ກວດສອບ, ທ�ານ
ຕ�ອງເປີດຟັ່ງຊັ້ນອອນລາຍ ເພື່ອໃຫ�ລະບົບເຮັດລາຍການບ�ອກເງິນໄວ�ຊົ່ວຄາວ 1 ໂດລາ, ແຕ�
ລາຍການດັ່ງກ�າວນີ້ຈະບ�່ຖືກຮຽກເກັບເຂົ້າບັດຂອງທ�ານ ແລະ ຈະຖືກປົດລ�ອກພາຍໃນ 10 ວັນ. 
• ປ�ອນຂ�້ມູນບັດ ແລະ ຕັ້ງ User: ປ�ອນຂ�້ມູນບັດໃຫ�ຖືກຕ�ອງ ເຊິ່ງຕ�ອງເປີດຟັ່ງຊັ່ນອອນລາຍໄວ�
ກ�ອນ 1 ຄັ້ງ). ຫຼັງຈາກນັ້່ນກົດ Verify . ຈາກນັ້ນ. ປ�ອນຂ�້ມູນສ�ວນຕົວ, ຊື່ ແລະ ນາມສະກຸນ, ເບີ...

            

---

## 💬 ໂໝດ Interactive - ພິມຄຳຖາມຂອງທ່ານ (ພິມ 'quit' ເພື່ອອອກ

---

👋 ຂອບໃຈທີ່ໃຊ້ລະບົບ RAG!
